In [ ]:
# 输入：tw_after_incident_0_10.xlsx和cl_news_sp500_20240105
# 输出：tw_after_incident_0_10_origin_highest_with_matches_update-all3.xlsx

In [ ]:
import pandas as pd

# 读取Excel文件
file_path = '/data1/dxw_data/llm/RA/hku_ivy/esg/cl_news_sp500_20240105.csv'  # 请替换为实际文件路径
df = pd.read_csv(file_path)

# 读取停词表
stopwords_file_path = '/data1/dxw_data/llm/ML/LIWC/datasets/stopwords_en.txt'
with open(stopwords_file_path, 'r') as file:
    stop_words = set(file.read().splitlines())

# 数据清洗函数
def clean_text(text):
    words = text.split()  # 简单分词
    cleaned_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(cleaned_words)

# 对"HD"列进行数据清洗
df['HD_cleaned'] = df['HD'].astype(str).apply(clean_text)

# 保存清洗后的数据到新Excel文件
output_file_path = '/data1/dxw_data/llm/RA/hku_ivy/esg/cleaned_data.xlsx'
df.to_excel(output_file_path, index=False)

print(f"Cleaned data saved to {output_file_path}")


In [ ]:
import pandas as pd
import re
from summa import summarizer
from tqdm import tqdm

# 读取Excel文件
file_path = '/data1/dxw_data/llm/RA/hku_ivy/esg/cleaned_data.xlsx'
df = pd.read_excel(file_path)

# 定义过滤和摘要函数
def clean_text(text):
    if isinstance(text, str):
        # 删除[]包裹的内容
        clean_text = re.sub(r'\[.*?\]', '', text)
        return clean_text
    return ''

def summarize_text(text):
    if isinstance(text, str) and text.strip() != '':
        # 使用summa.summarizer进行文本摘要
        summary = summarizer.summarize(text)
        return summary
    return ''

# 创建新列并初始化进度条
df['Cleaned_TXT'] = ''
df['Summary'] = ''
total = len(df)
progress_bar = tqdm(total=total, desc="Processing", unit="row")

# 对TXT列进行处理
for index, row in df.iterrows():
    try:
        cleaned_text = clean_text(row['TXT'])
        summary = summarize_text(cleaned_text)
        df.at[index, 'Cleaned_TXT'] = cleaned_text
        df.at[index, 'Summary'] = summary
    except Exception as e:
        print(f"Error processing row {index}: {e}")
    progress_bar.update(1)

# 关闭进度条
progress_bar.close()

# 保存到新的Excel文件中
new_file_path = '/data1/dxw_data/llm/RA/hku_ivy/esg/cleaned_data_update.xlsx'
df.to_excel(new_file_path, index=False)

print("文本处理和摘要已完成，并保存到cleaned_data_update.xlsx中。")


In [ ]:
import pandas as pd
from tqdm import tqdm

# 读取Excel文件
file_path = '/data1/dxw_data/llm/RA/hku_ivy/esg/cleaned_data_update.xlsx'
df = pd.read_excel(file_path)

# 初始化进度条
total = len(df)
progress_bar = tqdm(total=total, desc="Processing", unit="row")

# 对Summary列进行处理，将HD_cleaned列内容插入到Summary列的最前面
for index, row in df.iterrows():
    try:
        hd_cleaned_text = row['HD_cleaned']
        summary_text = row['Summary']
        
        if isinstance(hd_cleaned_text, str) and isinstance(summary_text, str):
            new_summary = f"{hd_cleaned_text}\n{summary_text}"
            df.at[index, 'Summary'] = new_summary
    except Exception as e:
        print(f"Error processing row {index}: {e}")
    progress_bar.update(1)

# 关闭进度条
progress_bar.close()

# 保存到新的Excel文件中
new_file_path = '/data1/dxw_data/llm/RA/hku_ivy/esg/cleaned_data_update_v2.xlsx'
df.to_excel(new_file_path, index=False)

print("Summary列更新已完成，并保存到cleaned_data_update_v2.xlsx中。")


In [ ]:
import pandas as pd

# Load the Excel file
file_path = '/data1/dxw_data/llm/RA/hku_ivy/esg/tw_after_incident_0_10.xlsx'
df = pd.read_excel(file_path)

# Function to select the row with the second-highest similarity_score in each group
def select_second_highest(group):
    if len(group) > 1:
        sorted_group = group.sort_values(by='similarity_score', ascending=False)
        return sorted_group.iloc[0]  # first highest
    else:
        return group.iloc[0]  # If only one row, return it

# Process each (company_ticker, incident_date) group independently
result = []
grouped = df.groupby(['company_ticker', 'incident_date'])

for _, group in grouped:
    selected_row = select_second_highest(group)
    result.append(selected_row)

# Concatenate all the selected rows
result_df = pd.DataFrame(result)

# Save the resulting DataFrame to a new file
output_file_path = '/data1/dxw_data/llm/RA/hku_ivy/esg/tw_after_incident_0_10_origin_highest.xlsx'
result_df.to_excel(output_file_path, index=False)

print("Selection completed and saved successfully.")


In [ ]:
# 这个是正确代码，全过程运行最后一版，直接运行就好。目前最新进度，只有一个问题，就是有些无法匹配的就留空了。
import pandas as pd
from datetime import datetime, timedelta
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import numpy as np
from joblib import Parallel, delayed
import torch

# 读取Excel文件
file1_path = '/data1/dxw_data/llm/RA/hku_ivy/esg/tw_after_incident_0_10_origin_highest.xlsx'
file2_path = '/data1/dxw_data/llm/RA/hku_ivy/esg/cleaned_data_update_v2.xlsx'

df1 = pd.read_excel(file1_path)
df2 = pd.read_excel(file2_path)

# 转换日期格式
df1['incident_date'] = pd.to_datetime(df1['incident_date'], format='%Y/%m/%d')
df2['PD'] = pd.to_datetime(df2['PD'], format='%Y-%m-%d')

# 初始化模型
model = SentenceTransformer('/data1/dxw_data/llm/paraphrase-multilingual-MiniLM-L12-v2')

# 创建新的列来保存匹配结果
for i in range(1, 6):
    df1[f'matched_summary_{i}'] = ""
    df1[f'PD_match_{i}'] = pd.NaT
    df1[f'similarity_{i}'] = 0.0

def process_row(idx1, row1, df2, model):
    try:
        incident_date_plus_15 = row1['incident_date'] + timedelta(days=15)
        matching_rows = df2[(df2['ticker'] == row1['company_ticker']) & 
                            (df2['PD'] >= row1['incident_date']) & 
                            (df2['PD'] <= incident_date_plus_15)]
        if matching_rows.empty:
            return idx1, None
        
        incident_words = row1['incident_category']
        incident_embeddings = [model.encode(word, convert_to_tensor=True) for word in incident_words]
        
        summary_sentences = matching_rows['Summary'].dropna().astype(str).str.split('\n').explode()
        summary_sentences = summary_sentences[summary_sentences.apply(lambda x: isinstance(x, str))]
        
        if summary_sentences.empty:
            return idx1, None
        
        summary_embeddings = model.encode(summary_sentences.tolist(), convert_to_tensor=True)
        
        total_similarities = []
        for summary_embedding in summary_embeddings:
            max_similarity = max(util.pytorch_cos_sim(incident_embedding, summary_embedding).item() for incident_embedding in incident_embeddings)
            total_similarities.append(max_similarity)
        
        summary_sentence_similarities = pd.Series(total_similarities, index=summary_sentences.index)
        
        matching_rows = matching_rows.join(summary_sentence_similarities.groupby(summary_sentence_similarities.index).sum().rename('total_similarity'))
        
        top_5_matching_rows = matching_rows.nlargest(5, 'total_similarity')
        
        results = []
        for i, row in enumerate(top_5_matching_rows.itertuples(), start=1):
            results.append((idx1, i, row.Summary, row.PD, row.total_similarity))
        
        return idx1, results
    except RuntimeError as e:
        if 'CUDA out of memory' in str(e):
            print(f"CUDA out of memory error at row {idx1}. Trying to free memory.")
            torch.cuda.empty_cache()
            return idx1, None
        else:
            raise e

# 提取需要计算相似度的行
valid_rows = [(idx1, row1, df2, model) for idx1, row1 in df1.iterrows()]

# 并行处理
results = Parallel(n_jobs=4)(delayed(process_row)(idx1, row1, df2, model) for idx1, row1, df2, model in tqdm(valid_rows, desc="Calculating similarities"))

# 处理结果
for idx1, res in results:
    if res is not None:
        for idx1, i, summary, pd_match, similarity in res:
            df1.at[idx1, f'matched_summary_{i}'] = summary
            df1.at[idx1, f'PD_match_{i}'] = pd_match
            df1.at[idx1, f'similarity_{i}'] = similarity

# 保存结果到新的Excel文件
output_file_path = '/data1/dxw_data/llm/RA/hku_ivy/esg/tw_after_incident_0_10_origin_highest_with_matches_update-all3.xlsx'
df1.to_excel(output_file_path, index=False)

print(f"结果已保存到 {output_file_path}")
